In [1]:
import itertools
import City_Preprocessing as bkPro
import pandas as pd
import numpy as np
pd.options.display.max_rows = 400
from tslearn.metrics import dtw
from sklearn.preprocessing import MinMaxScaler

In [2]:
city_raw = pd.read_csv('/Users/briankalinowski/PycharmProjects/CIS600/zecon/City_time_series.csv')

In [3]:
# Should ALWAYS have Date and RegionName
features = ['Date', 'RegionName',
                'PriceToRentRatio_AllHomes',
                'ZRI_AllHomes',
                'ZriPerSqft_AllHomes',
                'Zri_SingleFamilyResidenceRental']

# all our measurement Featreus
values = ['PriceToRentRatio_AllHomes',
                'ZRI_AllHomes',
                'ZriPerSqft_AllHomes',
                'Zri_SingleFamilyResidenceRental']

lookup_ix = ['State', 'City']
full_ix = ['State', 'City', 'Date']

city_rental_data = bkPro.process_city_data(city_raw, features, values, lookup_ix, full_ix)

/anaconda/envs/CIS600/lib/python3.7/site-packages/pandas/core/frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [4]:
city_rental_data.shape


(80820, 7)

In [5]:
city_rental_data.groupby(lookup_ix).ngroups

10132

In [6]:
city_rental_data.head()

,State,City,Date,AVG_PriceToRentRatio_AllHomes,AVG_ZRI_AllHomes,AVG_ZriPerSqft_AllHomes,AVG_Zri_SingleFamilyResidenceRental
0,PA,Doylestown Township,2010,15.443333,1910.666667,0.936000,2046.666667
1,PA,Doylestown Township,2011,14.130000,2082.833333,0.988333,2188.166667
2,PA,Doylestown Township,2012,13.661667,2177.833333,1.012833,2279.250000
3,PA,Doylestown Township,2013,14.012500,2151.666667,1.020333,2284.250000
4,PA,Doylestown Township,2014,14.093333,2169.083333,1.045333,2320.583333


In [7]:
city_rental_data.loc[city_rental_data['State'] == 'GENT_CONTROL'].loc[city_rental_data['City'] == 'GENT_CONTROL']

,State,City,Date,AVG_PriceToRentRatio_AllHomes,AVG_ZRI_AllHomes,AVG_ZriPerSqft_AllHomes,AVG_Zri_SingleFamilyResidenceRental
80812,GENT_CONTROL,GENT_CONTROL,2010,14.120115,1687.160920,1.236736,1731.850575
80813,GENT_CONTROL,GENT_CONTROL,2011,13.477414,1717.321839,1.261351,1763.370690
80814,GENT_CONTROL,GENT_CONTROL,2012,13.493534,1750.606322,1.290908,1793.359195
80815,GENT_CONTROL,GENT_CONTROL,2013,14.707213,1820.551724,1.342598,1859.735632
80816,GENT_CONTROL,GENT_CONTROL,2014,15.361609,1930.813218,1.414253,1967.186782
80817,GENT_CONTROL,GENT_CONTROL,2015,15.245862,2090.109195,1.539569,2135.922414
80818,GENT_CONTROL,GENT_CONTROL,2016,15.726494,2203.390805,1.629098,2245.747126
80819,GENT_CONTROL,GENT_CONTROL,2017,16.647184,2266.129310,1.679609,2303.793103


# DTW Test 

In [8]:
KNOWN_GENT = set(["Pittsburgh, PA", "San Diego, CA", "New York, NY", "Charleston, SC",
                  "Asheville, NC", "Washington, DC", "Portland, OR", "Denver, CO", "Nashville, TN",
                  "Sacramento, CA", "Jersey City, NJ", "Long Beach, CA", "Austin, TX", "Ann Arbor, MI",
                  "Bentonville, AR", "Thornton, CO", "Royal Oak, MI", "Costa Mesa, CA", "Berkeley, CA",
                  "Napa, CA", "Midland, TX", "Hillsboro, OR", "Oakland, CA", "Seattle, WA", "Vista, CA",
                  "Quincy, MA", "Hayward, CA", "San Marcos, CA", "Somerville, MA", "Los Angeles, CA", 
                  "Long Beach, CA", "Anaheim, CA"])


In [9]:
min_max_scaler = MinMaxScaler()
city_rental_data[['AVG_PriceToRentRatio_AllHomes',
           'AVG_ZRI_AllHomes',
           'AVG_ZriPerSqft_AllHomes',
           'AVG_Zri_SingleFamilyResidenceRental']] = pd.DataFrame(min_max_scaler.fit_transform(city_rental_data[[
                                                                 'AVG_PriceToRentRatio_AllHomes',
                                                                 'AVG_ZRI_AllHomes',
                                                                 'AVG_ZriPerSqft_AllHomes',
                                                                 'AVG_Zri_SingleFamilyResidenceRental']]))

city_rental_data.head()

,State,City,Date,AVG_PriceToRentRatio_AllHomes,AVG_ZRI_AllHomes,AVG_ZriPerSqft_AllHomes,AVG_Zri_SingleFamilyResidenceRental
0,PA,Doylestown Township,2010,0.380098,0.064320,0.113964,0.071601
1,PA,Doylestown Township,2011,0.340545,0.072383,0.124300,0.078313
2,PA,Doylestown Township,2012,0.326440,0.076832,0.129140,0.082634
3,PA,Doylestown Township,2013,0.337006,0.075606,0.130621,0.082871
4,PA,Doylestown Township,2014,0.339440,0.076422,0.135559,0.084594


In [18]:
# remove our GENT_CONTROL city/state from our city data
state_city_lookup = bkPro.get_city_state_lookup(city_rental_data, ['State', 'City'])
state_city_lookup.remove(('GENT_CONTROL', 'GENT_CONTROL'))

# dtw scores for each city when compared to our GENT_CONTROL average
dtw_dataframe = pd.DataFrame(columns=['State','City','dtw_score'])

# Gent. control Dataframe 
control_df = city_rental_data.loc[city_rental_data.State == 'GENT_CONTROL'].loc[city_rental_data.City == 'GENT_CONTROL']
control_df.drop(columns=['State', 'City', 'Date'], inplace=True)

for i in range(0, len(state_city_lookup)):
    # State/City names
    state = state_city_lookup[i][0]
    city = state_city_lookup[i][1]
    
    # locate each city and drop columns
    single_city_df = city_rental_data.loc[city_rental_data.State == state].loc[city_rental_data.City == city]
    single_city_df.drop(columns=['State', 'City', 'Date'], inplace=True)
    
    # calculate DTW score 
    dtw_score = dtw(single_city_df.values, control_df.values)
    
    # Append row to dtw df
    dtw_dataframe = dtw_dataframe.append({'State': state, 'City': city, 'dtw_score': dtw_score}, ignore_index=True)

In [19]:
dtw_dataframe.head()

,State,City,dtw_score
0,PA,Doylestown Township,0.264897
1,NJ,Forked River,0.295939
2,MO,Ash Grove,0.613447
3,MN,White Bear Lake,0.369573
4,GA,Lithonia,0.838989


In [27]:
dtw_dataframe.sort_values(by='dtw_score', ascending=True).head()

,State,City,dtw_score
8032,FL,Jupiter Island,3.905696
9051,CA,Atherton,2.849645
7754,CA,Beverly Hills,2.413206
8114,CA,Belvedere,2.314848
5127,FL,Palm Beach,2.299241


In [28]:
dtw_dataframe = dtw_dataframe.append({'State': 'GENT_CONTROL', 'City': 'GENT_CONTROL', 'dtw_score': 0.0}, ignore_index=True)

In [30]:
dtw_dataframe.sort_values(by='dtw_score', ascending=True).head()

,State,City,dtw_score
10131,GENT_CONTROL,GENT_CONTROL,0.000000
20,CA,Santee,0.064712
4495,CA,Rohnert Park,0.065260
7739,CA,Pinole,0.068630
9798,CA,Oceanside,0.070289


# DTW Encoding 

In [122]:
from sklearn.preprocessing import KBinsDiscretizer
from tslearn.preprocessing import TimeSeriesScalerMinMax
from tslearn.utils import to_time_series_dataset
from tslearn.piecewise import SymbolicAggregateApproximation

In [36]:
dtw_dataframe.shape

(10132, 3)

In [38]:
# Kbins
dis_encoder = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='kmeans')
dtw_arr = np.reshape(dtw_dataframe.dtw_score.values, (-1, 1))
kmeans_encoder = dis_encoder.fit_transform(dtw_arr)

In [39]:
unique, counts = np.unique(kmeans_encoder, return_counts=True)
dict(zip(unique, counts))

{0.0: 3264, 1.0: 4335, 2.0: 2490, 3.0: 41, 4.0: 2}

In [40]:
kmeans_encoder_df = pd.DataFrame(kmeans_encoder, columns=['dtw_value'])
dtw_values_dict = {0.0: 'HIGH_GENT', 1.0: 'MID_GENT', 2.0: 'NORMAL_GENT', 3.0: 'LOW_GENT', 4.0: 'NO_GENT'}
kmeans_encoder_df['dtw_label'] = kmeans_encoder_df.dtw_value.map(dtw_values_dict)


In [42]:
kmeans_encoder_df.shape

(10132, 2)

In [43]:
dtw_labeled = dtw_dataframe.merge(kmeans_encoder_df, how='outer', left_index=True, right_index=True)

In [45]:
dtw_labeled.sort_values(by='dtw_score', ascending=True).head()

#city_rental_data = city_rental_data.merge(dtw_labeled, on=['State', 'City'], how='left')

,State,City,dtw_score,dtw_value,dtw_label
10131,GENT_CONTROL,GENT_CONTROL,0.000000,0.0,HIGH_GENT
20,CA,Santee,0.064712,0.0,HIGH_GENT
4495,CA,Rohnert Park,0.065260,0.0,HIGH_GENT
7739,CA,Pinole,0.068630,0.0,HIGH_GENT
9798,CA,Oceanside,0.070289,0.0,HIGH_GENT


In [46]:
city_rental_data[city_rental_data.isnull().any(axis=1)]

,State,City,Date,AVG_PriceToRentRatio_AllHomes,AVG_ZRI_AllHomes,AVG_ZriPerSqft_AllHomes,AVG_Zri_SingleFamilyResidenceRental


In [50]:
city_rental_data = city_rental_data.merge(dtw_labeled, on=['State', 'City'], how='left')

In [51]:
city_rental_data.head()

,State,City,Date,AVG_PriceToRentRatio_AllHomes,AVG_ZRI_AllHomes,AVG_ZriPerSqft_AllHomes,AVG_Zri_SingleFamilyResidenceRental,dtw_score,dtw_value,dtw_label
0,PA,Doylestown Township,2010,0.380098,0.064320,0.113964,0.071601,0.264897,0.0,HIGH_GENT
1,PA,Doylestown Township,2011,0.340545,0.072383,0.124300,0.078313,0.264897,0.0,HIGH_GENT
2,PA,Doylestown Township,2012,0.326440,0.076832,0.129140,0.082634,0.264897,0.0,HIGH_GENT
3,PA,Doylestown Township,2013,0.337006,0.075606,0.130621,0.082871,0.264897,0.0,HIGH_GENT
4,PA,Doylestown Township,2014,0.339440,0.076422,0.135559,0.084594,0.264897,0.0,HIGH_GENT


# DTW Clustering

In [56]:
city_rental_data.head()

,State,City,Date,AVG_PriceToRentRatio_AllHomes,AVG_ZRI_AllHomes,AVG_ZriPerSqft_AllHomes,AVG_Zri_SingleFamilyResidenceRental,dtw_score,dtw_value,dtw_label
0,PA,Doylestown Township,2010,0.380098,0.064320,0.113964,0.071601,0.264897,0.0,HIGH_GENT
1,PA,Doylestown Township,2011,0.340545,0.072383,0.124300,0.078313,0.264897,0.0,HIGH_GENT
2,PA,Doylestown Township,2012,0.326440,0.076832,0.129140,0.082634,0.264897,0.0,HIGH_GENT
3,PA,Doylestown Township,2013,0.337006,0.075606,0.130621,0.082871,0.264897,0.0,HIGH_GENT
4,PA,Doylestown Township,2014,0.339440,0.076422,0.135559,0.084594,0.264897,0.0,HIGH_GENT


In [54]:
dtw_labeled.head()

,State,City,dtw_score,dtw_value,dtw_label
0,PA,Doylestown Township,0.264897,0.0,HIGH_GENT
1,NJ,Forked River,0.295939,0.0,HIGH_GENT
2,MO,Ash Grove,0.613447,1.0,MID_GENT
3,MN,White Bear Lake,0.369573,0.0,HIGH_GENT
4,GA,Lithonia,0.838989,2.0,NORMAL_GENT


In [160]:
from tslearn.clustering import TimeSeriesKMeans

In [131]:
city_rental_features = city_rental_data.drop(columns=['State', 'City', 'Date', 'dtw_score', 'dtw_value', 'dtw_label'])
city_rental_labels = city_rental_data.filter(['State', 'City', 'Date', 'dtw_score', 'dtw_value', 'dtw_label'])
city_rental_labels_dtw_mapping = city_rental_data.filter(['dtw_score', 'dtw_value', 'dtw_label'])
clustring_labels = city_rental_data.filter(['dtw_score'])
clustring_labels.head()


,dtw_score
0,0.264897
1,0.264897
2,0.264897
3,0.264897
4,0.264897


In [161]:
knn_ts = TimeSeriesKMeans(n_clusters=5, n_init=1, metric='dtw')
y_pred = knn_ts.fit_predict(city_rental_features.values)

0.008 --> 0.006 --> 0.005 --> 0.005 --> 0.005 --> 0.005 --> 0.005 --> 0.005 --> 0.005 --> 0.005 --> 0.005 --> 0.005 --> 0.005 --> 0.005 --> 0.005 --> 0.005 --> 0.005 --> 0.005 --> 0.005 --> 0.005 --> 0.005 --> 0.005 --> 0.005 --> 


In [163]:
unique, counts = np.unique(y_pred, return_counts=True)
dict(zip(unique, counts))

{0: 19888, 1: 20768, 2: 33288, 3: 850, 4: 6026}

In [166]:
unique, counts = np.unique(city_rental_data.dtw_value.values, return_counts=True)
dict(zip(unique, counts))

{0.0: 26013, 1.0: 34564, 2.0: 19899, 3.0: 328, 4.0: 16}

In [169]:
knn_ts_predict = pd.DataFrame(y_pred, columns=['dtw_cluster_prediction'])

In [172]:
knn_ts_predict.head()

,dtw_cluster_prediction
0,0
1,0
2,0
3,0
4,0


In [175]:
city_rental_data = city_rental_data.merge(knn_ts_predict, how='outer', left_index=True, right_index=True)

In [185]:
city_rental_data.head()

,State,City,Date,AVG_PriceToRentRatio_AllHomes,AVG_ZRI_AllHomes,AVG_ZriPerSqft_AllHomes,AVG_Zri_SingleFamilyResidenceRental,dtw_score,dtw_value,dtw_label,dtw_cluster_prediction
0,PA,Doylestown Township,2010,0.380098,0.064320,0.113964,0.071601,0.264897,0.0,HIGH_GENT,0.0
1,PA,Doylestown Township,2011,0.340545,0.072383,0.124300,0.078313,0.264897,0.0,HIGH_GENT,0.0
2,PA,Doylestown Township,2012,0.326440,0.076832,0.129140,0.082634,0.264897,0.0,HIGH_GENT,0.0
3,PA,Doylestown Township,2013,0.337006,0.075606,0.130621,0.082871,0.264897,0.0,HIGH_GENT,0.0
4,PA,Doylestown Township,2014,0.339440,0.076422,0.135559,0.084594,0.264897,0.0,HIGH_GENT,0.0


In [177]:
city_rental_data = city_rental_data.astype({'dtw_cluster_prediction': float})

In [186]:
city_rental_data.head()

,State,City,Date,AVG_PriceToRentRatio_AllHomes,AVG_ZRI_AllHomes,AVG_ZriPerSqft_AllHomes,AVG_Zri_SingleFamilyResidenceRental,dtw_score,dtw_value,dtw_label,dtw_cluster_prediction
0,PA,Doylestown Township,2010,0.380098,0.064320,0.113964,0.071601,0.264897,0.0,HIGH_GENT,0.0
1,PA,Doylestown Township,2011,0.340545,0.072383,0.124300,0.078313,0.264897,0.0,HIGH_GENT,0.0
2,PA,Doylestown Township,2012,0.326440,0.076832,0.129140,0.082634,0.264897,0.0,HIGH_GENT,0.0
3,PA,Doylestown Township,2013,0.337006,0.075606,0.130621,0.082871,0.264897,0.0,HIGH_GENT,0.0
4,PA,Doylestown Township,2014,0.339440,0.076422,0.135559,0.084594,0.264897,0.0,HIGH_GENT,0.0


In [187]:
city_rental_data.to_csv('/Users/briankalinowski/Desktop/CIS600_DataMining/Zillow_Data_Clean/city_data_DTW_predict.csv', 
                        index=None, header=True)